# A.8 Advanced Array Input and Output（数组输入和输出的高级用法）

第4章我们用np.save和np.load在磁盘上存储数组为二进制格式。其实还有很多其他方式，可以不用把数据写到RAM里。

# 1 Memory-Mapped Files（内存映射文件）

内存映射文件是一种用来与二进制文件交互的方法，尽管这些文件存储在磁盘上，但也能表现得像是存储在内存里一样。numpy里的memmap对象可以写入和读取大文件，而且不用把整个数组读取到内存里。另外，memmap有和内存数组一样的方法，因此可以直接使用ndarry的很多算法。

创建一个内存映射，使用np.memmap，传入一个file path（文件地址），dtype，shape，file mode（文件模式）：

In [2]:
import numpy as np

In [3]:
mmap = np.memmap('mymmap', dtype='float64', mode='w+', shape=(10000, 10000))

In [4]:
mmap

memmap([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

对memmap切片的话，返回数组在磁盘上的视图：

In [5]:
section = mmap[:5]

如果赋值的话，会缓存到内存里（就像一个python文件对象），但是我们可以使用flush直接写入磁盘：

In [6]:
section[:] = np.random.randn(5, 10000)

In [7]:
mmap.flush()

In [8]:
mmap

memmap([[ 1.60756953,  0.45694105, -1.17647715, ..., -0.84034465,
          0.1531578 , -1.45861771],
        [ 0.41242028, -0.04194728,  0.56079489, ..., -0.58545953,
         -0.93735441, -0.31906384],
        [-0.32320003,  1.94482684, -0.26242257, ..., -1.5787911 ,
          0.53292519, -0.69058973],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [9]:
del mmap

一旦内存映射超出范围，并且被垃圾回收（garbage-collected），任何修改都会被写入磁盘。当打开一个既存的内存映射，我们仍然需要指定dtype和shape，文件只是一些二进制数组块，不会有元数据在磁盘上（no metadata on disk）:

In [10]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))

In [11]:
mmap

memmap([[ 1.60756953,  0.45694105, -1.17647715, ..., -0.84034465,
          0.1531578 , -1.45861771],
        [ 0.41242028, -0.04194728,  0.56079489, ..., -0.58545953,
         -0.93735441, -0.31906384],
        [-0.32320003,  1.94482684, -0.26242257, ..., -1.5787911 ,
          0.53292519, -0.69058973],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

# 2 HDF5 and Other Array Storage Options（HDF5和其他一些数组存储选择）

PyTables和h5py都能存储GB甚至TB级别的数据，格式为HDF5，对于numpy交互也非常友好。